# Testando bibliotecas sgp4 + pymap3d 

**Links**

- https://www.scivision.co/pymap3d/html/index.html
- https://github.com/scivision/pymap3d

In [28]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from sgp4.earth_gravity import wgs84
from sgp4.io import twoline2rv
from datetime import datetime, timedelta
from datetime import timezone

from tle_getter import get_tle, SATELLITES_IDS

iss_tle = get_tle(name='ISS')

In [30]:
def propagate_satellite(satellite, at):
    date_at = at.year, at.month, at.day, at.hour, at.minute, at.second + (at.microsecond / 1000000.0)
    return satellite.propagate(*date_at)

In [31]:
satellite = twoline2rv(*iss_tle, wgs84)

In [32]:
at = datetime(year=2017, month=9, day=21, hour=1, minute=29, second=17, microsecond=23, tzinfo=timezone.utc)
r, v = propagate_satellite(satellite, at)
print('position = {}'.format(r))
print('velocity = {}'.format(v))

position = (-2971.9458295417226, -4140.43841805304, -4485.828318672087)
velocity = (3.4854753968182237, -6.006085589596043, 3.2330791835723622)


In [33]:
import pymap3d as pm

at = datetime.utcnow()
eci = r
fga_pos = (-15.989620, -48.044411, 1207)


res = pm.eci2aer(eci, *fga_pos, at)
print('res = {}'.format(res))

az, el, slant_range = res
print('(az, el) = ({}, {})'.format(az, el))

res = (array([187.50729063]), array([-89.8470762]), array([6380918.7505229]))
(az, el) = ([187.50729063], [-89.8470762])


In [44]:
from tle_getter import get_az_el


lat, lng, alt = fga_pos
norad_id = 25544

print(get_az_el(lat, lng, alt, norad_id=norad_id))

(249.73, -63.68)
